In [1]:
# IMPORT LIBRARIES #
import pandas as pd
import pygsheets
import numpy as np

In [2]:
# FLAGS #

#Set to 1 to print diagnostic info
debugFlag = 0
#Set to 1 to pull info from RawNames sheet, join GPS, update, and export
updateGPSFlag = 0

In [3]:
# IMPORT DATA #

gc = pygsheets.authorize(service_file='file.json')
sh = gc.open('DashData')

if debugFlag:
    for x in sh:
        print(x)

for x in sh:
    title = x.title
    if(title=="DeliveriesOld"):
        import_sheets_OldDelDF = x.get_as_df()
    elif(title=="Deliveries"):
        import_sheets_DelDF = x.get_as_df()
    elif(title=="Days"):
        import_sheets_DaysDF = x.get_as_df()
    elif(title=="Weeks"):
        import_sheets_WeeksDF = x.get_as_df()
    elif(title=="RawNames"):
        import_sheets_RawNamesDF = x.get_as_df()
    elif(title=="GPS"):
        #for import
        import_sheets_GPSDF = x.get_as_df()
        #for export
        import_wks_gps = x
    else:
        print("Unused worksheet detected: ", x)

Unused worksheet detected:  <Worksheet 'Import' index:6>
Unused worksheet detected:  <Worksheet 'Lifetime Deliveries' index:7>
Unused worksheet detected:  <Worksheet 'Days Old' index:8>
Unused worksheet detected:  <Worksheet 'Weeks Old' index:9>


In [281]:
# Allows for disjoint cells, one for import, one for reset to import value (for ease of testing)

#DF from sheets
sheets_OldDelDF   = import_sheets_OldDelDF.copy()
sheets_DelDF      = import_sheets_DelDF.copy()
sheets_DaysDF     = import_sheets_DaysDF.copy()
sheets_WeeksDF    = import_sheets_WeeksDF.copy()
sheets_RawNamesDF = import_sheets_RawNamesDF.copy()
sheets_GPSDF      = import_sheets_GPSDF.copy()

#Sheets, ah right can't use copy on a worksheet object, might need an alternate
#wks_gps           = import_wks_gps.copy()

In [282]:
# FUNCTIONS #

# TODO: develop function to take avg distance and duration for stacked orders
def processStacks(df):
    return df

def dollars_to_floats(df, columns):
    for x in columns:
        df[x] = df[x].str.replace('$', '')
        df[x] = df[x].str.replace(',', '')
        df[x] = df[x].astype(float)
        
def hours_to_floats(df, columns):
    for x in columns:
        for index, row in df.iterrows():
            active = row[x]
            activeH = float(active[:active.index(":")])
            activeM = round(float(active[active.index(":")+1:active.index(":")+3])/60, 2)
            active = activeH + activeM
            df.at[index, x] = active
        df[x] = df[x].astype(float)

def getStartHour(df):
    df["Start Hour"] = -1
    for index, row in df.iterrows():
        startHour = row["Start Time"]
        startHour = int(startHour[:startHour.index(":")])
        df.at[index, "Start Hour"] = startHour

def addStartAndEndDate(df):
    lastRow = df.iloc[-1]
    df = df[:-1]
    df["Start Date"] = -1
    df["End Date"]   = -1
    for index, row in df.iterrows():
        tempVar = row["Dates"]
        tempVar1 = tempVar[:tempVar.index(" ")]
        tempVar2 = tempVar[tempVar.index("- ")+2:]
        #print(tempVar1, tempVar2)
        df.at[index, "Start Date"] = tempVar1
        df.at[index, "End Date"] = tempVar2
    
    print(df.dtypes)
    #print(list(df.columns.values))
    df = pd.concat([df, lastRow.to_frame().T], ignore_index=True)
    df.reset_index(drop=True, inplace=True)
    #print(list(df.columns.values))
    print(df.dtypes)
    return df

#Currently only valid for 2023
def date_to_datetime(df, column):
    for index, row in df.iterrows():
        df.at[index, column] = pd.Timestamp('2023/' + row[column])
    df[column] = pd.to_datetime(df[column])
    #print("TEST")
    #print(df.dtypes)
        
        
#Currently only valid for 2023
# Currently unused function
# Would we actually want this to have different dates? it's really the date the shift started
# So maybe, keep date same, but certain analysis might require that difference to be handled
def time_to_timestamp(df, DateCol, TimeCols):
    for index, row in df.iterrows():
        df.at[index, TimeCols[0]] = pd.Timestamp('2023/' + row[DateCol] + " " + row[TimeCols[0]])
        df.at[index, TimeCols[1]] = pd.Timestamp('2023/' + row[DateCol] + " " + row[TimeCols[1]])

In [283]:
# UPDATE GPS WORKSHEET #

if updateGPSFlag:
    testingDF = sheets_RawNamesDF.copy()
    testingDF = testingDF.drop_duplicates(subset=["Restaurant Name", "RAW DATA"])
    testingDF = testingDF[["Restaurant Name", "RAW DATA", "Note"]]
    newNamesDF = testingDF.merge(sheets_GPSDF, how='left')
    for index, row in newNamesDF.iterrows():
        if isinstance(row['Address'], float):
            if not row['Note'] == "":
                newNamesDF.at[index, "Address"] = row["Note"]
            else:
                print("Note not found for:", index)
        if (row['Address'] == ""):
            newNamesDF.at[index, "Resolved"] = "0"
        else:
            newNamesDF.at[index, "Resolved"] = "1"
    newNamesDF = newNamesDF.drop(columns=["Note"])
    import_wks_gps.set_dataframe(newNamesDF,(1,1))

In [284]:
# START PROCESSING #
#sheets_result.head()

In [285]:
# COMBINE DELIVERY DATA #
#sheets_frames = [sheets_OldDelDF, sheets_DelDF]
#sheets_result = pd.concat(sheets_frames)
#sheets_result = sheets_OldDelDF.append(sheets_DelDF)
#sheets_result = pd.concat([sheets_OldDelDF, sheets_DelDF], sort=False)
#sheets_result.reset_index(drop=True, inplace=True)

In [286]:
# FUNCTIONS AND OTHER SHORT PROCESSING STEPS #


#All
dollars_to_floats(sheets_OldDelDF, ["Total", "Base", "Tip", "Peak Bonus"])
#V3 only
dollars_to_floats(sheets_DelDF, ["Total", "Base", "Tip", "Peak Bonus"])
getStartHour(sheets_DelDF)
#Days
dollars_to_floats(sheets_DaysDF, ["Total", "Base", "Tip"])
hours_to_floats(sheets_DaysDF, ["Dash", "Active"])
date_to_datetime(sheets_DaysDF, "Date")
sheets_DaysDF["Day of Week"] = sheets_DaysDF["Date"].dt.day_name()
#Weeks
dollars_to_floats(sheets_WeeksDF, ["Total", "Base", "Tip", "Adj pay", "Other", "Pre Total"])
hours_to_floats(sheets_WeeksDF, ["Dash", "Active"])
#print(sheets_WeeksDF.dtypes)
sheets_WeeksDF = addStartAndEndDate(sheets_WeeksDF)
# pretotalactiveratio refers to weekly pre adjustment pay / time spent on deliveries
#print(sheets_WeeksDF.dtypes)
sheets_WeeksDF["pretotalactiveratio"] = round(sheets_WeeksDF["Pre Total"]/sheets_WeeksDF["Active"], 2)

/tmp/ipykernel_30941/3616702342.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')


Dates              object
Base              float64
Tip               float64
Adj pay           float64
Other             float64
Pre Total         float64
Total             float64
Active            float64
Dash              float64
Deliveries          int64
Total Mileage      object
Active Mileage     object
Version            object
WeekID             object
dtype: object
Dates              object
Base              float64
Tip               float64
Adj pay           float64
Other             float64
Pre Total         float64
Total             float64
Active            float64
Dash              float64
Deliveries          int64
Total Mileage      object
Active Mileage     object
Version            object
WeekID             object
Start Date         object
End Date           object
dtype: object
Dates             object
Base              object
Tip               object
Adj pay           object
Other             object
Pre Total         object
Total             object
Active           

/tmp/ipykernel_30941/3616702342.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')
/tmp/ipykernel_30941/3616702342.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')
/tmp/ipykernel_30941/3616702342.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Start Date"] = -1
/tmp/ipykernel_30941/3616702342.py:34: SettingWithCopyWarning: 
A value is trying to be set on 

TypeError: loop of ufunc does not support argument 0 of type float which has no callable rint method

In [ ]:
sheets_WeeksDF

In [ ]:
# BEGIN ADJ PAY SECTION #

# This establishes the estimated pay adjustment estimates by delivery, date, and week
# This is calculated using time spent delivering and distance approximations
# The real adjusted pay is based on weekly values
# I suspect the primary cause for difference between estimate and real values stems from 
#how I estimate mileage vs how the app tracks mileage

# DFs are copied to avoid excluding dropped values in later analysis



#if debugFlag:
#    sheets_DelDF.tail()

In [ ]:
#Per delivery estimations
#sheets_DelDF1 = sheets_DelDF.copy()

#Convert to float if possible, else convert to NaN
#sheets_DelDF1["Duration"] = pd.to_numeric(sheets_DelDF1["Duration"], errors='coerce')
sheets_DelDF["Duration"] = pd.to_numeric(sheets_DelDF["Duration"])
#Drop NaN rows
#sheets_DelDF1 = sheets_DelDF1[sheets_DelDF1["Duration"].notna()]
#Calculate estimate adjustment per delivery
sheets_DelDF["Est Adj"] = (sheets_DelDF["Duration"]/60*18
                           + sheets_DelDF["Distance"]*(0.34) ) - sheets_DelDF["Base"]
#Duration is over 60 because its value is in minutes, values from day and week are in hours

#if debugFlag:
#    sheets_DelDF.tail()

In [ ]:
#if debugFlag:
#    sheets_DaysDF.tail()

In [ ]:
#Per day estimations
sheets_DaysDF1 = sheets_DaysDF.copy()
sheets_DaysDF1 = sheets_DaysDF1[sheets_DaysDF1["Version"] == 3]
sheets_DaysDF1["Est Adj"] = (sheets_DaysDF1["Active"]*(18) 
                            + sheets_DaysDF1["Active Mileage"]*(0.34) ) - sheets_DaysDF1["Base"]

In [ ]:
#if debugFlag:
#    sheets_DaysDFv3.tail()

In [ ]:
#Per week estimations
sheets_WeeksDF1 = sheets_WeeksDF.copy()
sheets_WeeksDF1 = sheets_WeeksDF1[sheets_WeeksDF1["Version"] == 3] #totals doesn't have v3 so its dropped
sheets_WeeksDF1["Est Adj"] = (sheets_WeeksDF1["Active"]*(18) 
                            + sheets_WeeksDF1["Active Mileage"]*(0.34) ) - sheets_WeeksDF1["Base"]

In [ ]:
#appends


# Sample values
if debugFlag:
    print("Del DF Sample\n")
    print(sheets_DelDF.iloc[-1])
    print("\nDays DF Sample\n")
    print(sheets_DaysDF1.iloc[-1])
    print("\nWeeks DF Sample\n")
    print(sheets_WeeksDF1.iloc[-1])
    
    
# END ADJ PAY SECTION #

In [ ]:
print(list(sheets_DelDF.columns.values))
print()
print(list(sheets_DaysDF.columns.values))
print()
print(list(sheets_WeeksDF.columns.values))
print()
print(sheets_DelDF["Start Hour"].unique())

In [ ]:
sheets_WeeksDF.iloc[-1]

In [ ]:
#ACTUALLY LETS DO THE RESULTS JOIN HERE

sheets_result = pd.concat([sheets_OldDelDF, sheets_DelDF], sort=False)
sheets_result.reset_index(drop=True, inplace=True)

#NO okay so we have to concat df1 (v3) to df (v1,2), THIS SHOULD WORK
sheets_DaysDF0 = sheets_WeeksDF[sheets_WeeksDF["Version"] != 3]
sheets_WeeksDF0 = sheets_WeeksDF[sheets_WeeksDF["Version"] != 3] #THIS PUTS TOTALS IN THE MIDDLE
total_S = sheets_WeeksDF0.iloc[-1]
sheets_WeeksDF0 = sheets_WeeksDF0[:-1]
#okay so what if we grab wksdf0.iloc-1, store, drop, then add it at the end

sheets_DaysDF = pd.concat([sheets_DaysDF0, sheets_DaysDF1], sort=False)
sheets_DaysDF.reset_index(drop=True, inplace=True)

sheets_WeeksDF = pd.concat([sheets_WeeksDF0, sheets_WeeksDF1], sort=False)
sheets_WeeksDF = pd.concat([sheets_WeeksDF, total_S.to_frame().T], ignore_index=True)
sheets_WeeksDF.reset_index(drop=True, inplace=True)



In [ ]:
tempVar = sheets_WeeksDF["Dates"].iloc[0]
tempVar1 = tempVar[:tempVar.index(" ")]
tempVar2 = tempVar[tempVar.index("- ")+2:]
print(tempVar1, tempVar2)

In [ ]:
# END PROCESSING #
# START INITIAL EXPLORATION #

In [ ]:
#Percentage of deliveries taken that tip
numUntipped = (sheets_result["Tip"] == 0.0).sum()
numTipped = (sheets_result["Tip"] != 0.0).sum()
percTipped = round((numTipped/(numUntipped+numTipped)), 4)
print(percTipped*100, "% of accepted deliveries included tip")

#Average tip, including and excluding untipped deliveries
TipAvg = round(sheets_result["Tip"].mean(), 2)
print("$", TipAvg, "average tip value including orders with no tip")
TipAvg2 = round(sheets_result[sheets_result["Tip"] !=0]["Tip"].mean(), 2)
print("$", TipAvg2, "average tip value excluding orders with no tip")

# Overall hourly rate
hourlyRate = round(sheets_WeeksDF.iloc[-1]["Total"]/sheets_WeeksDF.iloc[-1]["Dash"], 2)
print("Overall hourly pay rate is $", hourlyRate)

# Max and Average values by Delivery, Day, and Week
# Some of these values are wrong, because I have not yet implemented a stack value handler
print()
print("Daily Max Values")
print(sheets_DaysDF[["Total", "Base", "Tip", "Active", "Dash", "Deliveries"]].max())
print(sheets_DaysDF[sheets_DaysDF["Version"] == 3][["Total Mileage", "Active Mileage"]].max(), "\n" )
print("Daily Average Values")
print(sheets_DaysDF[["Total", "Base", "Tip", "Active", "Dash", "Deliveries"]].mean())
print(sheets_DaysDF[sheets_DaysDF["Version"] == 3][["Total Mileage", "Active Mileage"]].mean(), "\n" )
print()
# Some of these values are wrong, because I have not yet implemented a stack value handler
print("Weekly Max Values")
#
print(sheets_WeeksDF[["Base", "Tip", "Adj pay", "Pre Total", "Total", "Active", "Dash", "Deliveries"]].max())
#
print(sheets_WeeksDF[sheets_WeeksDF["Version"] == 3][["Total Mileage", "Active Mileage"]].max(), "\n")
print("Weekly Average Values")
#
print(sheets_WeeksDF[["Base", "Tip", "Adj pay", "Pre Total", "Total", "Active", "Dash", "Deliveries"]].mean())
#
print(sheets_WeeksDF[sheets_WeeksDF["Version"] == 3][["Total Mileage", "Active Mileage"]].mean(), "\n")
print()
# Some of these values are wrong, because I have not yet implemented a stack value handler
print("Delivery Max Values")
#
print(sheets_DelDF[["Distance", "Duration"]].max())
print(sheets_result[["Total"]].max(), "\n")
print("Delivery Average Values")
#
print(sheets_DelDF[["Distance", "Duration"]].mean())
print(sheets_result[["Total"]].mean(), "\n")

# END INITIAL EXPLORATION #

In [ ]:
# Restaurants rankable by avg or mean base/tip/total
# Yes, total is almost always base+tip, but total INCLUDES peak pay, which may be desirable information

tempDF = sheets_result.copy()
#The line below drops restaurants that only appear once, we may no longer want this to happen
#Commented out for now, may make sense to use when considering outliers for averages
#tempDF = tempDF[sheets_result["Restaurant Name"].duplicated(keep=False) == True]
tempDF = tempDF.groupby(["Restaurant Name"]).agg({"Base": [np.mean, np.sum], 
                                                  "Tip": [np.mean, np.sum],
                                                  "Total": [np.mean, np.sum],
                                                 'Restaurant Name': 'size'})
tempDF.columns = ['Avg Base', 'Sum Base', 'Avg Tip', 
                  'Sum Tip', 'Avg Total', 'Sum Total', "Deliveries"]

tempDF = tempDF.round({'Avg Base': 2, 
                       'Sum Base': 2,
                       'Avg Tip': 2,
                       'Sum Tip': 2,
                       'Avg Total': 2,
                       'Sum Total': 2})

#Change sort value to see differences
tempDF1 = tempDF.index.copy()
restaurantsDF = tempDF.copy()
tempDF.sort_values("Sum Total", ascending=False)







#RESTAURANTS DF

In [ ]:
#Dollars Earned vs Miles Driven is another factor I wanted to consider in the overall earnings equation

# Grab v3 deliveries, drop rows without known distance values
# Group by restaurant name, sum on total and distance
# Evaluate total pay over total distance as new column
# Sort low to high



#WE COULD GRAB RESULTS.VERSION==3 INSTEAD 
tempDF = sheets_result[sheets_result["Version"] == 3].copy()
#tempDF = sheets_DelDF.copy()
#tempDF = tempDF[tempDF.Distance!="-"]
#tempDF["Distance"] = tempDF["Distance"].astype(float)
tempDF = tempDF.groupby(["Restaurant Name"]).agg({"Total": "sum", "Distance": "sum"})
tempDF = tempDF.round({'Base': 2, 'Tip': 2})
tempDF["DollarMilesRatio"] = tempDF["Total"]/tempDF["Distance"]
#tempDF.sort_values(by="DollarMilesRatio")
tempDF2 = tempDF.index.copy()

restaurantsDF = restaurantsDF.join(tempDF["DollarMilesRatio"])

tempDF


#JOIN WITH RESTAURANTS DF


#160 in total, 115 in v3? so 45 restaurants unique to v1 and 2??

#Can I select results where version !=3, get rest count name no
#can we select rest from temp1 not in temp2?

In [ ]:
restaurantsDF

In [ ]:
tempDF3 = tempDF1[~tempDF1.isin(tempDF2)].dropna(how = 'all')
tempDF3 #These are the restaurants from just v1, v2
#Thought there were too many but, there were actually a lot that haven't been seen since v3

In [ ]:
# Average $/hour and tip amount by day of week
# NOTE: This excludes pay adjustments

tempDF = sheets_DaysDF.copy()

# Can move this up to formatting function calls
# Also, check if necessary to convert from string to timestamp to datetime
#date_to_timestamp(tempDF, "Date")
#tempDF["Date"] = pd.to_datetime(tempDF["Date"])

#Adds column to DaysDF for the day of the week (Monday, Thursday, etc.)
#tempDF["Day of Week"] = tempDF["Date"].dt.day_name()

tempDF = tempDF.groupby(["Day of Week"]).agg(
    {"Dash": "sum", 
     "Total": [np.mean, np.sum], 
     "Tip": "sum", 
     "Deliveries": "sum"})
tempDF.columns = ['Sum Dash', 'Avg Daily Total', 'Sum Total', 'Sum Tip', 'Sum Deliveries']

tempDF["$/hr"] = tempDF["Sum Total"]/tempDF["Sum Dash"]
tempDF["Avg Delivery Tip"] = tempDF["Sum Tip"]/tempDF["Sum Deliveries"]
tempDF["Avg Delivery Total"] = tempDF["Sum Total"]/tempDF["Sum Deliveries"]

tempDF = tempDF.round({'Sum Dash': 2, 
                       'Avg Total': 2, 
                       'Sum Total':2, 
                       'Sum Tip': 2,
                       '$/hr': 2,
                       'AvgTip': 2,
                       'AvgTotal':2})

display(tempDF)